In [2]:
from src.dataset.Moabb2BGenerator_One_Person import Moabb2BGenerator
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import ConcatDataset

from src.dataset.MI_dataset_all_subjects import MI_Dataset as MI_Dataset_all_subjects
from src.dataset.MI_dataset_single_subject import MI_Dataset as MI_Dataset_single_subject

from config.default import cfg


from models.eegnet import EEGNet

from utils.eval import accuracy

%load_ext autoreload
%autoreload 2



<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
d:\OE\Tantargyak\Agyi_Jelek\PyTorch-NeuroMotor-master\.venv\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


In [4]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
channels =  3
samples =  401
num_classes_2b = 2 

model = EEGNet(channels=channels, samples=samples, num_classes=num_classes_2b)

## Loading data for the 2b model

In [5]:
train_data_generator = Moabb2BGenerator(subject_id=1,runs=[0,1,2],train_data=True, device=device)
X_2b, y_2b = train_data_generator.X, train_data_generator.y

train_dataset_2b = TensorDataset(X_2b, y_2b)
train_loader_2b = DataLoader(train_dataset_2b, batch_size=cfg['train']['batch_size'], shuffle=True, drop_last=True)


Before:  (3, 48, 3, 401)
After:  torch.Size([144, 3, 401])


In [6]:
test_data_generator = Moabb2BGenerator(subject_id=1,runs=[3,4],train_data=False, device=device)
test_X_2b, test_y_2b = test_data_generator.X, test_data_generator.y

test_dataset_2b = TensorDataset(test_X_2b, test_y_2b)
test_loader_2b = DataLoader(test_dataset_2b, batch_size=cfg['train']['batch_size'], shuffle=True, drop_last=True)

Before:  (2, 48, 3, 401)
After:  torch.Size([96, 3, 401])


In [7]:
print(f"Train dataset: {len(train_data_generator)} samples")

for features, label in train_loader_2b:
    print(features.shape)
    print(label)
    break

Train dataset: 144 samples
torch.Size([48, 3, 401])
tensor([1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0])


In [8]:
# Test forward pass
model(next(iter(train_data_generator))[0]);

In [9]:
for features, labels in train_loader_2b:
    outputs = model(features)
    _, predicted = torch.max(outputs.data, 1)
    print(labels.shape, predicted.shape)  # Ellenőrizze a dimenziókat
    break


torch.Size([48]) torch.Size([48])


## Training 2b dataset

In [11]:
# Finomhangolás
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

finetune_epochs =  300

for epoch in range(finetune_epochs):
    epoch_loss = 0.0
    for X_batch, y_batch in train_loader_2b:      
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

if epoch % 10 == 9:
        train_accuracy = accuracy(model, train_loader_2b)
        test_accuracy = accuracy(model, test_loader_2b)
        print(f"Epoch {epoch + 1}/{cfg['train']['n_epochs']}, Loss: {epoch_loss}, Train accuracy: {train_accuracy:.2f}%, Test accuracy: {test_accuracy:.2f}%")


print("#"*50)
print(f'Final_loss: {epoch_loss}')
print(f'Final train accuracy: {accuracy(model, train_loader_2b):.2f}%')
print(f'Final test accuracy: {accuracy(model, test_loader_2b):.2f}%')

torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])
torch.Size([48, 3, 401])


In [12]:
torch.save(model.state_dict(), 'model_state_2b_single_subject.pth')

## Loading data for 2a model

In [7]:
subject = 1
train_runs = [0,1,2,3,4]
test_runs = [5]


train_dataset = MI_Dataset_single_subject(subject, train_runs, device = device)
test_dataset = MI_Dataset_single_subject(subject, test_runs, device = device)

train_dataloader = DataLoader(train_dataset,  batch_size=cfg['train']['batch_size'], shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset,  batch_size=cfg['train']['batch_size'], shuffle=False, drop_last=True)

25
25


In [8]:
print(f"Train dataset: {len(train_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")

for features, label in train_dataloader:
    print(features.shape)
    print(label.shape)
    break

Train dataset: 240 samples
Test dataset: 48 samples
torch.Size([48, 3, 401])
torch.Size([48])


In [19]:
# Function to load 2b model weights for 2a
def load_model(model_path: str, channels: int, samples: int, num_classes: int) -> torch.nn.Module:
    # EEGNet példányosítása
    model = EEGNet(channels=channels, samples=samples, num_classes=num_classes)

    # Modell súlyainak betöltése, kihagyva csak az utolsó dense réteget
    model_weights = torch.load(model_path, map_location=device)
    model_weights = {k: v for k, v in model_weights.items() if 'dense' not in k}
    model.load_state_dict(model_weights, strict=False)


    in_features = model.dense.in_features  # Az eredeti réteg bemeneti jellemzőinek száma
    model.dense = nn.Linear(in_features, 4)  

    model.to(device)
    return model


In [20]:
channels = train_dataset.channels
samples = train_dataset.time_steps

model_2a = load_model("model_state_2b_single_subject.pth",channels=channels,samples=samples, num_classes=4)

In [15]:
summary(model_2a, input_size=(5, 10, *next(iter(train_dataloader))[0][0].shape));

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 3, 402]           1,024
       BatchNorm2d-2           [-1, 16, 3, 402]              32
            Conv2d-3           [-1, 32, 1, 402]              96
       BatchNorm2d-4           [-1, 32, 1, 402]              64
               ELU-5           [-1, 32, 1, 402]               0
         AvgPool2d-6           [-1, 32, 1, 100]               0
           Dropout-7           [-1, 32, 1, 100]               0
            Conv2d-8           [-1, 32, 1, 101]             512
            Conv2d-9           [-1, 32, 1, 101]           1,024
      BatchNorm2d-10           [-1, 32, 1, 101]              64
              ELU-11           [-1, 32, 1, 101]               0
        AvgPool2d-12            [-1, 32, 1, 12]               0
          Dropout-13            [-1, 32, 1, 12]               0
          Flatten-14                  [

In [17]:
# Test forward pass
model(next(iter(train_dataloader))[0]);

## Training dataset 2a using 2b weights

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2a.parameters(), lr=cfg['train']['learning_rate'], weight_decay=cfg['train']['weight_decay'])

# Training loop
for epoch in range(cfg['train']['n_epochs']):
    epoch_loss = 0.0

    for batch_features, batch_labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model_2a(batch_features)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if epoch % 10 == 9:
        train_accuracy = accuracy(model_2a, train_dataloader)
        test_accuracy = accuracy(model_2a, test_dataloader)
        print(f"Epoch {epoch + 1}/{cfg['train']['n_epochs']}, Loss: {epoch_loss}, Train accuracy: {train_accuracy:.2f}%, Test accuracy: {test_accuracy:.2f}%")

print("#"*50)
print(f'Final_loss: {epoch_loss}')
print(f'Final train accuracy: {accuracy(model_2a, train_dataloader):.2f}%')
print(f'Final test accuracy: {accuracy(model_2a, test_dataloader):.2f}%')

Epoch 10/200, Loss: 5.190884947776794, Train accuracy: 50.00%, Test accuracy: 58.33%
Epoch 20/200, Loss: 4.463633716106415, Train accuracy: 60.83%, Test accuracy: 37.50%
Epoch 30/200, Loss: 4.118259429931641, Train accuracy: 68.75%, Test accuracy: 50.00%
Epoch 40/200, Loss: 3.9460121989250183, Train accuracy: 67.50%, Test accuracy: 43.75%
Epoch 50/200, Loss: 3.675355315208435, Train accuracy: 76.25%, Test accuracy: 39.58%
Epoch 60/200, Loss: 3.557062089443207, Train accuracy: 75.00%, Test accuracy: 45.83%
Epoch 70/200, Loss: 3.3338372111320496, Train accuracy: 74.17%, Test accuracy: 37.50%
Epoch 80/200, Loss: 3.220353662967682, Train accuracy: 76.25%, Test accuracy: 41.67%
Epoch 90/200, Loss: 3.1593871116638184, Train accuracy: 75.83%, Test accuracy: 47.92%
Epoch 100/200, Loss: 3.1444273591041565, Train accuracy: 78.75%, Test accuracy: 33.33%
Epoch 110/200, Loss: 2.9143782258033752, Train accuracy: 77.92%, Test accuracy: 35.42%
Epoch 120/200, Loss: 2.648120790719986, Train accuracy: 78

In [22]:
torch.save(model.state_dict(), 'model_state_2a_transfer_learning_single_subject.pth')